# Multiprocessing, Threads, subprocess, concurrent.futures, asyncio, event loops...

### There are lots of names

#### (thread, task, process...)

### There are a lot of options

### Options for what?

* Parallel (concurrency)
    * Many tasks at same time
    * Utilize full CPU
* Process control (run scripts and stuff)
    * Control command-line scripts from Python
    * Control Python from Python


Some things are taken from this [good article](https://realpython.com/python-concurrency/).

### Parallel

Reasons for using parallel:

* Problem creates or requires too much data for a normal.
* Sun would explode before the computation would finish.
* Algorithm is easy to parallelize.
* The physics itself cannot be simulated with smaller resources.


Source: [Effective Computation in Physics](http://shop.oreilly.com/product/0636920033424.do)

### Embarrassingly Parallel

Tasks are independent from each other but need to do a lot of them.

* Summing a very large array

* Monte Carlo simulations
* Stochastic & Genetic algorithms

### _More processors means more fasterer_

* Should say "processes" not "processors".
* Should say "faster" not "more fasterer".

### Non-embarrassingly Parallel

* Canonical example: inverting a matrix

### Large-scale problems defined by architecture

(nothing to do with Python)

* High-performance computing (HPC)
    * aka Supercomputers
* High-throughput computing (HTC
    * aka Distributed Computing

## CPU-Bound vs I/O-Bound

* Waiting on the CPU (difficult tasks)
-vs-
* Waiting on a slow device (network, hard-drive, hardware).

_**Opening and closing files is the most expensive thing you can do**_

(i.e. you are I/O-bound)

# Python

Two big players:

* Threads
* Multiprocessing


`concurrent.futures` is part of standard library and makes using either one easy.

# Threads

## **Don't**

## Multiprocessing

## Gotchas

* Need to have a `__main__`.  [Read why](https://docs.python.org/3/library/multiprocessing.html#multiprocessing-programming).
* Sharing information between processes (use [`Queues` and `Pipes`](https://docs.python.org/3/library/multiprocessing.html#exchanging-objects-between-processes)).

## Examples